In [34]:
# -*- coding: utf8 -*-

In [35]:
debug=False

filePrefix="MGC-171229"
nomLexique="/Users/gilles/Copy/Python/phonemisation/bdlexique-PDM-Resync2.txt"
nomLexique="/Volumes/gilles/Transfert/Copies-iMac-GB/Python/phonemisation/bdlexique-PDM-Resync3.txt"
freqForme=1000000

## Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [36]:
import pandas as pd
import pickle

# Préparation du tableau de VERBES

### Lecture du lexique
- nomLexique pour le fichier
- names pour les noms de colonnes
- élimination des lignes dupliquées éventuelles (p.e. dépendre)

In [37]:
bdlexique=pd.read_csv(nomLexique,sep=";",names=["ortho","phono","ext","cs","ms","vs","lexeme","L23","lemmeFilms","lemmeFrantext","formeFilms","formeFrantext","baseFilms","baseFrantext"],encoding="utf8")
bdlexique.loc[(bdlexique["ms"]=="ij")&(bdlexique["cs"].isin(["V","K"])),"ms"]="MS"
bdlexique.drop_duplicates(inplace=True,subset=["phono","ext","cs","ms","vs","lexeme","lemmeFilms","lemmeFrantext","formeFilms","formeFrantext","baseFilms","baseFrantext"])

### Extraction des formes verbales dans une structure *verbes*

In [38]:
verbes=bdlexique[bdlexique["cs"].isin(["V","K"])].copy()
verbes.loc[verbes["ms"]=="ij","ms"]="MS"

### Création de la colonne de fréquence *freq*

- fréquence forme : 1000000x(formeFrantext+formeFilms)
- fréquence lexeme : lemmeFrantext+lemmeFilms
- fréquence inexistante : .01*freqBaseCase

In [39]:
verbes["freqLemme"]=verbes["lemmeFilms"].astype(float)+verbes["lemmeFrantext"].astype(float)
verbes["freqForme"]=verbes["formeFilms"].astype(float)+verbes["formeFrantext"].astype(float)
verbes["freqBase"]=verbes["baseFilms"].astype(float)+verbes["baseFrantext"].astype(float)
verbes["freq"]=verbes["freqForme"]*freqForme+verbes["freqLemme"]+0.01*verbes["freqBase"]

### Elimination des colonnes inutiles
- indicateur L23
- fréquences brutes

In [40]:
inutilesColonnes=["L23","lemmeFrantext","lemmeFilms","formeFrantext","formeFilms",
                  "baseFilms","baseFrantext","freqLemme","freqForme","freqBase"]
for colonne in inutilesColonnes:
    del verbes[colonne]

### Ajout d'une colonne *prob* pour la fréquence relative

In [41]:
verbes["prob"]=verbes["freq"]/verbes["freq"].sum()

### Ajout d'une colonne *case*
- remplir les *ms* vides pour permettre la concaténation sans erreur
- remplir les *vs* vides des participes passés par "pp" pour permettre l'identification de la case
- *case* est la concaténation de *vs* et *ms*

In [42]:
verbes["ms"]=verbes["ms"].fillna("")
verbes["vs"]=verbes["vs"].fillna("pp")
verbes["case"]=verbes["vs"]+verbes["ms"]

In [43]:
bdlexiqueIn = unicode(u"èò")
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO")
neutralise = dict(zip(bdlexiqueNum, neutreOut))

def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

verbes["phono"]=verbes["phono"].apply(recoder)

In [44]:
with open(filePrefix+'-Verbes3.pkl', 'wb') as output:
   pickle.dump(verbes, output, pickle.HIGHEST_PROTOCOL)

In [45]:
verbes

,ortho,phono,ext,cs,ms,vs,lexeme,freq,prob,case
40,a,a,NaN,V,3S,pi,avoir,7.167735e+11,2.076214e-02,pi3S
45,abaissa,abEsa,NaN,V,3S,ai,abaisser,2.640001e+08,7.647056e-06,ai3S
46,abaissai,abEsE,NaN,V,1S,ai,abaisser,1.700001e+07,4.924243e-07,ai1S
47,abaissaient,abEsE,"t""",V,3P,ii,abaisser,4.100005e+07,1.187612e-06,ii3P
48,abaissais,abEsE,"z""",V,1S,ii,abaisser,4.933548e+01,1.429057e-12,ii1S
49,abaissais,abEsE,"z""",V,2S,ii,abaisser,4.954692e-01,1.435182e-14,ii2S
50,abaissait,abEsE,"t""",V,3S,ii,abaisser,2.520003e+08,7.299467e-06,ii3S
52,abaissant,abEsâ,"t""",V,,pP,abaisser,2.510001e+08,7.270495e-06,pP
56,abaissas,abEsa,"z""",V,2S,ai,abaisser,2.565900e-01,7.432415e-15,ai2S
57,abaissasse,abEsas,@,V,1S,is,abaisser,3.093462e-01,8.960556e-15,is1S
